# 빅콘테스트 퓨처스리그 : KBO 정규시즌 팀별 승률, 타율, 방어율 예측

팀별 월별 경기 구장

# Crawling

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import lxml

In [4]:
year = [2016, 2017, 2018, 2019, 2020]
month = ['03', '04', '05', '06', '07', '08', '09', '10']

In [29]:
table_year = pd.DataFrame()

for i in year:
    table_month = pd.DataFrame()
    for j in month:
        url = 'https://sports.news.naver.com/kbaseball/schedule/index.nhn?month={}&year={}'.format(j, i)
        req = requests.get(url)
        html = req.text
        table = pd.read_html(html)
        for z in range(1, len(table)):
            table_m = table[z]
            table_m['year'] = i
            table_month = table_month.append(table_m)
    table_year = table_year.append(table_month)

In [30]:
table_year

,0,1,2,year,3,4,5,6
0,3.1 (화),-,프로야구 경기가 없습니다.,2016,NaN,NaN,NaN,NaN
0,3.2 (수),-,프로야구 경기가 없습니다.,2016,NaN,NaN,NaN,NaN
0,3.3 (목),-,프로야구 경기가 없습니다.,2016,NaN,NaN,NaN,NaN
0,3.4 (금),-,프로야구 경기가 없습니다.,2016,NaN,NaN,NaN,NaN
0,3.5 (토),-,프로야구 경기가 없습니다.,2016,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
0,10.28 (수),-,프로야구 경기가 없습니다.,2020,NaN,NaN,NaN,NaN
0,10.29 (목),-,프로야구 경기가 없습니다.,2020,NaN,NaN,NaN,NaN
0,10.30 (금),-,프로야구 경기가 없습니다.,2020,NaN,NaN,NaN,NaN
0,10.31 (토),-,프로야구 경기가 없습니다.,2020,NaN,NaN,NaN,NaN


In [31]:
table_year.columns = ['날짜', '시간', '경기','year', '중계/기록', '중계방송사', '구장', '알림받기']
table_year

,날짜,시간,경기,year,중계/기록,중계방송사,구장,알림받기
0,3.1 (화),-,프로야구 경기가 없습니다.,2016,NaN,NaN,NaN,NaN
0,3.2 (수),-,프로야구 경기가 없습니다.,2016,NaN,NaN,NaN,NaN
0,3.3 (목),-,프로야구 경기가 없습니다.,2016,NaN,NaN,NaN,NaN
0,3.4 (금),-,프로야구 경기가 없습니다.,2016,NaN,NaN,NaN,NaN
0,3.5 (토),-,프로야구 경기가 없습니다.,2016,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
0,10.28 (수),-,프로야구 경기가 없습니다.,2020,NaN,NaN,NaN,NaN
0,10.29 (목),-,프로야구 경기가 없습니다.,2020,NaN,NaN,NaN,NaN
0,10.30 (금),-,프로야구 경기가 없습니다.,2020,NaN,NaN,NaN,NaN
0,10.31 (토),-,프로야구 경기가 없습니다.,2020,NaN,NaN,NaN,NaN


In [32]:
df = table_year.copy()
df = df[df.경기 != '프로야구 경기가 없습니다.']
df = df[df.중계방송사 != '해당 경기는 현지 사정으로 취소되었습니다.']
df.drop(['시간', '중계/기록', '중계방송사', '알림받기'], axis=1, inplace=True)
df = df.dropna()

In [33]:
df.reset_index(drop=True, inplace=True) # 인덱스 리셋
df

,날짜,경기,year,구장
0,3.8 (화),두산 5:5 kt,2016,수원
1,3.8 (화),SK 6:6 롯데,2016,울산
2,3.8 (화),삼성 5:3 NC,2016,마산
3,3.8 (화),넥센 2:4 한화,2016,대전
4,3.9 (수),LG 3:0 KIA,2016,광주
...,...,...,...,...
4496,10.18 (일),KIA VS LG,2020,잠실
4497,10.18 (일),KT VS SK,2020,문학
4498,10.18 (일),롯데 VS NC,2020,창원
4499,10.18 (일),두산 VS 키움,2020,고척


In [35]:
# 월 컬럼 생성
month = []
day = []
for i in range(len(df['날짜'])):
    string = df.loc[i, '날짜']
    pattern = re.compile('\(.\)')
    ls = re.sub(pattern, '', string).split('.')
    month.append(int(ls[0].strip()))
    day.append(int(ls[1].strip()))

df['month'] = month
df['day'] = day

In [36]:
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

In [37]:
df

,날짜,경기,year,구장,month,day
0,3.8 (화),두산 5:5 kt,2016,수원,3,8
1,3.8 (화),SK 6:6 롯데,2016,울산,3,8
2,3.8 (화),삼성 5:3 NC,2016,마산,3,8
3,3.8 (화),넥센 2:4 한화,2016,대전,3,8
4,3.9 (수),LG 3:0 KIA,2016,광주,3,9
...,...,...,...,...,...,...
4496,10.18 (일),KIA VS LG,2020,잠실,10,18
4497,10.18 (일),KT VS SK,2020,문학,10,18
4498,10.18 (일),롯데 VS NC,2020,창원,10,18
4499,10.18 (일),두산 VS 키움,2020,고척,10,18


In [38]:
df.reset_index(drop = True, inplace = True)

In [39]:
df['경기'].unique()

array(['두산  5:5  kt', 'SK  6:6  롯데', '삼성  5:3  NC', ..., '롯데  VS NC',
       '두산  VS 키움', '삼성  VS 한화'], dtype=object)

In [40]:
# 텍스트 전처리 함수 선언
def cleanText(string):
    pattern = re.compile('경기취소')
    string = re.sub(pattern, '', string)
    pattern = re.compile('\d+:\d+')
    string = re.sub(pattern, 'VS', string)
    return string

In [41]:
# 왼쪽 팀, 오른쪽 팀 컬럼 생성
team_lft = []
team_rht = []
for i in range(len(df['경기'])):
    a = df.loc[i, '경기']
    a = cleanText(a)
    teams = a.split('VS')
    team_lft.append(teams[0].strip())
    team_rht.append(teams[1].strip())

# 데이터프레임에 컬럼 추가
df['team_visit'] = team_lft
df['team_home'] = team_rht

# 경기 컬럼 삭제
df.drop('경기', axis=1, inplace=True)

In [42]:
# 컬럼 순서 변경
df = df.reindex(columns=['year', 'month', 'day', 'team_visit', 'team_home', '구장'])

In [43]:
df

,year,month,day,team_visit,team_home,구장
0,2016,3,8,두산,kt,수원
1,2016,3,8,SK,롯데,울산
2,2016,3,8,삼성,NC,마산
3,2016,3,8,넥센,한화,대전
4,2016,3,9,LG,KIA,광주
...,...,...,...,...,...,...
4496,2020,10,18,KIA,LG,잠실
4497,2020,10,18,KT,SK,문학
4498,2020,10,18,롯데,NC,창원
4499,2020,10,18,두산,키움,고척


In [44]:
df.to_csv('../../data/kbo_schedule_stadium.csv')